# Progetto Machine Learning: Riconoscimento di Specie di Uccelli con CNN

Questo notebook implementa un sistema di riconoscimento di specie di uccelli attraverso l'analisi di registrazioni audio della competizione BirdClef 2025. Il progetto utilizza un'architettura CNN per classificare gli audio convertiti in spettrogrammi Mel e include anche un sistema di configurazione automatica dell'ambiente per eseguire il codice su Kaggle, Google Colab o in locale.

## 1. Importazione delle Librerie Necessarie

Importiamo tutte le librerie necessarie per l'elaborazione audio, deep learning e visualizzazione.

In [ ]:
# Librerie di sistema e utilità
import os
import sys
import platform
import time
import warnings
import logging
import datetime
from pathlib import Path
import pprint as pp
import seaborn as sns
from collections import Counter
import IPython.display as ipd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# Sostituisci le importazioni di Transformers con timm
import timm

# Librerie per data science e manipolazione dati
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Librerie per elaborazione audio
import librosa
import librosa.display

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Visualizzazione
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Ignoriamo i warning
warnings.filterwarnings("ignore")

# Configurazione del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('BirdClef')

print("Librerie importate con successo!")
print(f"PyTorch versione: {torch.__version__}")
print(f"timm versione: {timm.__version__}")
print(f"Python versione: {platform.python_version()}")
print(f"Sistema operativo: {platform.system()} {platform.release()}")

In [ ]:
import shutil
import os

# Imposta questo a True per abilitare la cancellazione
clear_working_dir = True

working_dir = '/kaggle/working/'

if clear_working_dir:
    for filename in os.listdir(working_dir):
        file_path = os.path.join(working_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # elimina file o link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # elimina directory
        except Exception as e:
            print(f'Errore durante la rimozione di {file_path}: {e}')
    print(f"Tutti i file in {working_dir} sono stati rimossi.")
else:
    print("Pulizia disabilitata (clear_working_dir = False)")


## 2. Configurazione dell'Ambiente di Esecuzione

In questa sezione configuriamo l'ambiente di esecuzione in modo che il notebook funzioni sia su Kaggle, che su Google Colab, che in locale. Il codice rileverà automaticamente l'ambiente e configurerà i percorsi di conseguenza.

In [ ]:
# Variabile per impostare manualmente l'ambiente
# Modifica questa variabile in base all'ambiente in uso:
# - 'kaggle' per l'ambiente Kaggle
# - 'colab' per Google Colab
# - 'local' per l'esecuzione in locale
MANUAL_ENVIRONMENT = ''  # Impostare su 'kaggle', 'colab', o 'local' per forzare l'ambiente

def detect_environment():
    """
    Rileva se il notebook è in esecuzione su Kaggle, Google Colab o in locale.
    Rispetta l'impostazione manuale se fornita.
    
    Returns:
        str: 'kaggle', 'colab', o 'local'
    """
    # Se l'ambiente è stato impostato manualmente, usa quello
    if MANUAL_ENVIRONMENT in ['kaggle', 'colab', 'local']:
        print(f"Utilizzo ambiente impostato manualmente: {MANUAL_ENVIRONMENT}")
        return MANUAL_ENVIRONMENT
    
    # Verifica Kaggle con metodo più affidabile
    # Verifica l'esistenza di directory specifiche di Kaggle
    if os.path.exists('/kaggle/working') and os.path.exists('/kaggle/input'):
        print("Rilevato ambiente Kaggle")
        return 'kaggle'
    
    # Verifica se è Google Colab
    try:
        import google.colab
        return 'colab'
    except ImportError:
        pass
    
    # Se non è né Kaggle né Colab, allora è locale
    return 'local'

# Rileva l'ambiente attuale
ENVIRONMENT = detect_environment()
print(f"Ambiente rilevato: {ENVIRONMENT}")

In [ ]:
class Config:
    def __init__(self):
        # Rileva l'ambiente
        self.environment = ENVIRONMENT  # Usa la variabile globale impostata in precedenza
        
        # Imposta i percorsi di base in base all'ambiente
        if self.environment == 'kaggle':
            self.COMPETITION_NAME = "birdclef-2025"
            self.BASE_DIR = f"/kaggle/input/{self.COMPETITION_NAME}"
            self.OUTPUT_DIR = "/kaggle/working"
            self.MODELS_DIR = "/kaggle/input"  # Per i modelli pre-addestrati
            
            # Imposta subito i percorsi derivati per l'ambiente Kaggle
            self._setup_derived_paths()
            
        elif self.environment == 'colab':
            # In Colab, inizializza directory base temporanee
            self.COMPETITION_NAME = "birdclef-2025"
            self.OUTPUT_DIR = "/content/output"
            self.MODELS_DIR = "/content/models"
            
            # Crea le directory di output
            os.makedirs(self.OUTPUT_DIR, exist_ok=True)
            os.makedirs(self.MODELS_DIR, exist_ok=True)
            
            # In Colab, BASE_DIR verrà impostato dopo il download
            # quindi non impostiamo ancora i percorsi derivati
            self.BASE_DIR = "/content/placeholder"  # Verrà sovrascritto dopo il download
            
            # Inizializza i percorsi dei file a None per ora
            self.TRAIN_AUDIO_DIR = None
            self.TEST_SOUNDSCAPES_DIR = None
            self.TRAIN_CSV_PATH = None
            self.TAXONOMY_CSV_PATH = None
            self.SAMPLE_SUB_PATH = None
            
        else:  # locale
            # In ambiente locale, i percorsi dipenderanno dalla tua configurazione
            self.BASE_DIR = os.path.abspath(".")
            self.OUTPUT_DIR = os.path.join(self.BASE_DIR, "output")
            self.MODELS_DIR = os.path.join(self.BASE_DIR, "models")
            
            # Crea le directory se non esistono
            os.makedirs(self.OUTPUT_DIR, exist_ok=True)
            os.makedirs(self.MODELS_DIR, exist_ok=True)
            
            # Imposta i percorsi derivati
            self._setup_derived_paths()
        
        # Parametri per il preprocessing audio - già allineati con vincitori
        self.SR = 32000      # Sample rate
        self.DURATION = 5    # Durata dei clip in secondi
        self.N_MELS = 128    # Numero di bande Mel
        self.N_FFT = 1024    # Dimensione finestra FFT
        self.HOP_LENGTH = 500  # Hop length per STFT
        self.FMIN = 40       # Frequenza minima per lo spettrogramma Mel
        self.FMAX = 15000    # Frequenza massima
        self.POWER = 2       # Esponente per calcolo spettrogramma
            
        # Parametri per il training - aggiornati secondo i vincitori
        self.BATCH_SIZE = 96  # Aumentato da 32 a 96 come dai vincitori
        self.EPOCHS = 10     # Numero di epoche per il training
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        self.NUM_WORKERS = 4  # Aumentato per migliorare il data loading

        # Parametri per inference/submission
        self.TEST_CLIP_DURATION = 5  # Durata dei segmenti per la predizione (secondi)
        self.N_CLASSES = 0  # Sarà impostato dopo aver caricato i dati

    def _setup_derived_paths(self):
        """Imposta i percorsi derivati basati su BASE_DIR"""
        # Utilizza la normale divisione di percorso di OS (non il backslash hardcoded)
        self.TRAIN_AUDIO_DIR = os.path.join(self.BASE_DIR, "train_audio")
        self.TEST_SOUNDSCAPES_DIR = os.path.join(self.BASE_DIR, "test_soundscapes")
        self.TRAIN_CSV_PATH = os.path.join(self.BASE_DIR, "train.csv")
        self.TAXONOMY_CSV_PATH = os.path.join(self.BASE_DIR, "taxonomy.csv") 
        self.SAMPLE_SUB_PATH = os.path.join(self.BASE_DIR, "sample_submission.csv")

In [ ]:
config = Config()

# Gestione download dati in Colab con kagglehub
if config.environment == 'colab':
    # Percorsi nella cache di kagglehub
    cache_competition_path = "/root/.cache/kagglehub/competitions/birdclef-2025"
    cache_model_path = "/root/.cache/kagglehub/models/maurocarlu/simplecnn/PyTorch/default/1"
    cache_model_file = os.path.join(cache_model_path, "baseline_bird_cnn_model_val.pth")
    
    # Verifica se i dati sono già presenti nella cache
    data_exists = os.path.exists(os.path.join(cache_competition_path, "train.csv"))
    model_exists = os.path.exists(cache_model_file)
    
    if data_exists and model_exists:
        print("I dati e il modello sono già presenti nella cache. Utilizzo copie esistenti.")
        birdclef_path = cache_competition_path
        model_path = cache_model_path
    else:
        print("Scaricamento dati con kagglehub...")
        
        try:
            import kagglehub
            
            # Scarica solo i dati della competizione se necessario
            if not data_exists:
                print("Download dataset...")
                kagglehub.login()  # Mostra dialog di login interattivo
                birdclef_path = kagglehub.competition_download('birdclef-2025')
            else:
                print("Dataset già presente nella cache.")
                birdclef_path = cache_competition_path
                
            # Scarica solo il modello se necessario
            if not model_exists:
                print("Download modello...")
                kagglehub.login()  # Potrebbe essere necessario riautenticarsi
                model_path = kagglehub.model_download('maurocarlu/simplecnn/PyTorch/default/1')
            else:
                print("Modello già presente nella cache.")
                model_path = cache_model_path
                
            print(f"Download completato.")
            
        except Exception as e:
            print(f"Errore durante il download dei dati: {e}")
            print("Prova ad usare Google Drive o esegui su Kaggle.")
            
            # Se il download fallisce ma i dati esistono parzialmente, usa quelli
            if os.path.exists(cache_competition_path):
                birdclef_path = cache_competition_path
                print(f"Usando i dati esistenti in: {birdclef_path}")
            if os.path.exists(cache_model_path):
                model_path = cache_model_path
                print(f"Usando il modello esistente in: {model_path}")
    
    # Aggiorna i percorsi nella configurazione
    config.BASE_DIR = birdclef_path
    config._setup_derived_paths()
    config.MODELS_DIR = model_path
    model_file = os.path.join(model_path, "baseline_bird_cnn_model_val.pth")
    
    print(f"Dati disponibili in: {config.BASE_DIR}")
    print(f"Modello disponibile in: {model_file}")

# Stampa percorsi aggiornati
print(f"\nPercorso file CSV di training: {config.TRAIN_CSV_PATH}")
print(f"Percorso directory audio di training: {config.TRAIN_AUDIO_DIR}")

## 3. Configurazione del Modello e Parametri

Definiamo i parametri di configurazione per il preprocessamento audio, la creazione dello spettrogramma Mel e l'addestramento della CNN.

In [ ]:
# I parametri principali sono già definiti nella classe Config
# Verifichiamo l'esistenza delle directory e creiamo quelle necessarie per l'output

def setup_output_directories():
    """
    Configura le directory per l'output del progetto.
    
    Returns:
        dict: Dictionary con i percorsi delle directory di output
    """
    # Directory principale di output
    output_dir = config.OUTPUT_DIR
    
    # Sotto-directory per diversi tipi di output
    dirs = {
        'checkpoints': os.path.join(output_dir, 'checkpoints'),
        'tensorboard': os.path.join(output_dir, 'tensorboard_logs'),
        'predictions': os.path.join(output_dir, 'predictions'),
        'submissions': os.path.join(output_dir, 'submissions'),
        'visualizations': os.path.join(output_dir, 'visualizations'),
    }
    
    # Crea tutte le directory
    for dir_name, dir_path in dirs.items():
        os.makedirs(dir_path, exist_ok=True)
        print(f"Directory '{dir_name}' creata/verificata in: {dir_path}")
    
    return dirs

# Configura le directory di output
output_dirs = setup_output_directories()

# Crea un file di log per tenere traccia dei risultati
log_file_path = os.path.join(config.OUTPUT_DIR, f"experiment_log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

with open(log_file_path, 'w') as log_file:
    log_file.write(f"=== BirdClef Experiment Log ===\n")
    log_file.write(f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.write(f"Environment: {config.environment}\n\n")
    log_file.write("Output directories:\n")
    for dir_name, dir_path in output_dirs.items():
        log_file.write(f"- {dir_name}: {dir_path}\n")

print(f"File di log creato in: {log_file_path}")

# Memorizziamo i parametri di configurazione principali per l'addestramento
print("\nParametri di configurazione principali:")
print(f"- Sample rate: {config.SR} Hz")
print(f"- Durata clip audio: {config.DURATION} secondi")
print(f"- Numero bande Mel: {config.N_MELS}")
print(f"- Dimensione FFT: {config.N_FFT}")
print(f"- Hop length: {config.HOP_LENGTH}")
print(f"- Device: {config.DEVICE}")
print(f"- Batch size: {config.BATCH_SIZE}")
print(f"- Epoche: {config.EPOCHS}")

## 4. Caricamento e Preprocessing dei Dati

In questa sezione carichiamo i metadati dal file CSV di training, creiamo codifiche one-hot per le etichette delle specie e implementiamo funzioni per il caricamento e preprocessamento dei file audio.

In [ ]:
# Caricamento dei metadati
def load_metadata():
    """
    Carica e prepara i metadati dal file CSV di training.
    
    Returns:
        tuple: training_df, all_species, labels_one_hot
    """
    print(f"Caricamento metadati da: {config.TRAIN_CSV_PATH}")
    train_df = pd.read_csv(config.TRAIN_CSV_PATH)
    sample_sub_df = pd.read_csv(config.SAMPLE_SUB_PATH)
    
    # Estrai tutte le etichette uniche
    train_primary_labels = train_df['primary_label'].unique()
    train_secondary_labels = set([lbl for sublist in train_df['secondary_labels'].apply(eval) 
                                 for lbl in sublist if lbl])
    submission_species = sample_sub_df.columns[1:].tolist()  # Escludi row_id
    
    # Combina tutte le possibili etichette
    all_species = sorted(list(set(train_primary_labels) | train_secondary_labels | set(submission_species)))
    N_CLASSES = len(all_species)
    config.N_CLASSES = N_CLASSES  # Aggiorna il numero di classi nella configurazione
    
    print(f"Numero totale di specie trovate: {N_CLASSES}")
    print(f"Prime 10 specie: {all_species[:10]}")
    
    # Crea mappatura etichette-indici
    species_to_int = {species: i for i, species in enumerate(all_species)}
    int_to_species = {i: species for species, i in species_to_int.items()}
    
    # Aggiungi indici numerici al dataframe
    train_df['primary_label_int'] = train_df['primary_label'].map(species_to_int)
    
    # Prepara target multi-etichetta
    mlb = MultiLabelBinarizer(classes=all_species)
    mlb.fit(None)  # Fit con tutte le classi
    
    def get_multilabel(row):
        labels = eval(row['secondary_labels'])  # Valuta la lista di stringhe in modo sicuro
        labels.append(row['primary_label'])
        return list(set(labels))  # Assicura etichette uniche
    
    train_df['all_labels'] = train_df.apply(get_multilabel, axis=1)
    train_labels_one_hot = mlb.transform(train_df['all_labels'])
    
    print(f"Forma delle etichette one-hot: {train_labels_one_hot.shape}")
    
    return train_df, all_species, train_labels_one_hot, species_to_int, int_to_species

# Carica i metadati
train_df, all_species, train_labels_one_hot, species_to_int, int_to_species = load_metadata()

# Suddividi i dati in training e validation
def split_data(train_df, labels_one_hot, test_size=0.2, random_state=42):
    """
    Suddivide il dataset in set di training e validation.
    
    Args:
        train_df: DataFrame con i metadati
        labels_one_hot: Array di etichette one-hot
        test_size: Percentuale dei dati da usare per validation
        random_state: Seed per riproducibilità
        
    Returns:
        tuple: X_train_df, X_val_df, y_train_one_hot, y_val_one_hot
    """
    # Indici per lo split
    train_indices, val_indices = train_test_split(
        range(len(train_df)),
        test_size=test_size,
        random_state=random_state
    )
    
    # Crea i dataframe e gli array di etichette splittati
    X_train_df = train_df.iloc[train_indices].reset_index(drop=True)
    X_val_df = train_df.iloc[val_indices].reset_index(drop=True)
    
    y_train_one_hot = labels_one_hot[train_indices]
    y_val_one_hot = labels_one_hot[val_indices]
    
    print(f"Dimensioni Training Set: {X_train_df.shape}, Etichette: {y_train_one_hot.shape}")
    print(f"Dimensioni Validation Set: {X_val_df.shape}, Etichette: {y_val_one_hot.shape}")
    
    return X_train_df, X_val_df, y_train_one_hot, y_val_one_hot

# Suddividi i dati in training e validation
X_train_df, X_val_df, y_train_one_hot, y_val_one_hot = split_data(train_df, train_labels_one_hot)

    
# Per Kaggle, dovremo creare un dataset speciale per le soundscapes di test
# Questo verrà utilizzato direttamente nella fase di generazione della submission
# Non creiamo X_test_df e test_dataset per ora
X_test_df = None
y_test_one_hot = None

## Funzione di bilanciamento del dataset - Cancella una percentuale di esempi dalle classi molto numerose

In [ ]:
def create_balanced_dataset_df(train_df, labels_one_hot, abundant_class_threshold=200, remove_percentage=0.3, random_state=42):
    """
    Crea un DataFrame bilanciato rimuovendo parte degli esempi con rating bassi dalle classi abbondanti.
    
    Args:
        train_df: DataFrame originale
        labels_one_hot: Array di etichette one-hot
        abundant_class_threshold: Soglia per definire una classe come "abbondante"
        remove_percentage: Percentuale di esempi con rating 1-3 da rimuovere dalle classi abbondanti
        random_state: Seed per riproducibilità
        
    Returns:
        tuple: (DataFrame bilanciato, etichette one-hot bilanciate)
    """
    # Conta esempi per ogni classe
    class_counts = train_df['primary_label'].value_counts()
    
    # Identifica classi abbondanti
    abundant_classes = class_counts[class_counts > abundant_class_threshold].index.tolist()
    print(f"Classi identificate come abbondanti (>{abundant_class_threshold} esempi): {len(abundant_classes)}")
    
    # Copia il DataFrame originale
    balanced_df = train_df.copy()
    rows_to_drop = []
    
    # Contatori per statistiche
    total_removed = 0
    removed_by_class = {}
    
    # Per ogni classe abbondante
    for cls in abundant_classes:
        # Filtra esempi con rating 1-3 per questa classe
        low_quality_mask = (balanced_df['primary_label'] == cls) & (balanced_df['rating'].isin([1, 2, 3]))
        low_quality_indices = balanced_df[low_quality_mask].index.tolist()
        
        # Numero di esempi da rimuovere
        n_to_remove = int(len(low_quality_indices) * remove_percentage)
        
        # Seleziona casualmente gli indici da rimuovere
        np.random.seed(random_state)
        if n_to_remove > 0:
            indices_to_remove = np.random.choice(low_quality_indices, size=n_to_remove, replace=False)
            
            # Memorizza gli indici da rimuovere
            rows_to_drop.extend(indices_to_remove)
            
            # Aggiorna statistiche
            removed_by_class[cls] = n_to_remove
            total_removed += n_to_remove
    
    # Rimuovi le righe selezionate
    if rows_to_drop:
        balanced_df = balanced_df.drop(rows_to_drop).reset_index(drop=True)
        
        # Aggiorna anche le etichette one-hot rimuovendo gli stessi indici
        mask = np.ones(len(train_df), dtype=bool)
        mask[rows_to_drop] = False
        balanced_labels = labels_one_hot[mask]
    else:
        balanced_labels = labels_one_hot
    
    # Statistiche finali
    print(f"Totale esempi rimossi: {total_removed} ({total_removed/len(train_df):.1%} del dataset originale)")
    print(f"Dimensione dataset originale: {len(train_df)}")
    print(f"Dimensione dataset bilanciato: {len(balanced_df)}")
    
    # Visualizza le prime 5 classi con maggiori rimozioni
    if removed_by_class:
        top_removed = sorted(removed_by_class.items(), key=lambda x: x[1], reverse=True)[:5]
        print("\nClassi con maggior numero di esempi rimossi:")
        for cls, count in top_removed:
            original = class_counts[cls]
            remaining = original - count
            print(f"- {cls}: {count} rimossi, {remaining}/{original} rimanenti ({remaining/original:.1%})")
    else:
        print("Nessun esempio rimosso.")
    
    return balanced_df, balanced_labels

## 4.5 Analisi Esplorativa dei Dati (EDA)

In questa sezione esploreremo le caratteristiche del dataset per comprendere meglio la distribuzione delle specie, le proprietà audio e identificare eventuali pattern nei dati.

In [ ]:
# Configurazione stile visualizzazioni
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid", font_scale=1.1)
plt.rcParams['figure.figsize'] = [12, 6]

print("=== Statistiche di base del dataset ===")
print(f"Numero totale di registrazioni: {len(train_df)}")
print(f"Numero di specie uniche nel dataset: {len(all_species)}")
print(f"Campi disponibili nei metadati: {train_df.columns.tolist()}")

# Verifichiamo i dati mancanti
missing_data = train_df.isnull().sum()
print("\n=== Valori mancanti ===")
print(missing_data[missing_data > 0])

# 1. Distribuzione delle specie nel dataset (visualizzazione migliorata)
print("\n=== Analisi delle Specie ===")
primary_species_count = train_df['primary_label'].value_counts()

# Plot combinato: distribuzione delle specie con evidenza delle classi rare
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# A sinistra: top 20 specie più rappresentate
sns.barplot(x=primary_species_count.head(20).index, y=primary_species_count.head(20).values, ax=ax1)
ax1.set_title('Top 20 Specie per Numero di Registrazioni')
ax1.set_xlabel('Specie')
ax1.set_ylabel('Numero di Registrazioni')
ax1.tick_params(axis='x', rotation=90)

# A destra: distribuzione del numero di esempi per specie
sns.histplot(primary_species_count, bins=30, kde=True, ax=ax2)
ax2.set_title('Distribuzione del Numero di Registrazioni per Specie')
ax2.set_xlabel('Numero di Registrazioni')
ax2.set_ylabel('Conteggio Specie')
ax2.axvline(x=primary_species_count.median(), color='r', linestyle='--', 
            label=f'Mediana: {primary_species_count.median()}')
ax2.axvline(x=primary_species_count.mean(), color='g', linestyle='--', 
            label=f'Media: {primary_species_count.mean():.1f}')
ax2.axvline(x=50, color='orange', linestyle=':', label='Soglia classi rare (50)')
ax2.legend()

plt.tight_layout()
plt.show()

# Calcolo dell'indice di Gini per misurare lo sbilanciamento
def gini_coefficient(x):
    x = np.sort(x)
    n = len(x)
    index = np.arange(1, n+1)
    return (np.sum((2*index - n - 1) * x)) / (n * np.sum(x))

gini = gini_coefficient(primary_species_count.values)
print(f"\nIndice di Gini per la distribuzione delle specie: {gini:.4f}")
print(f"Questo indica {'un alto' if gini > 0.6 else 'un moderato' if gini > 0.3 else 'un basso'} livello di sbilanciamento nel dataset.")

# 2. NUOVA ANALISI: Rating per le classi con pochi esempi
# Definisco la soglia per le classi rare (< 50 esempi)
RARE_CLASS_THRESHOLD = 50
rare_species = primary_species_count[primary_species_count < RARE_CLASS_THRESHOLD].index.tolist()
print(f"\n=== Analisi dei Rating per Classi Rare (<{RARE_CLASS_THRESHOLD} esempi) ===")
print(f"Numero di classi rare: {len(rare_species)} su {len(all_species)} totali ({len(rare_species)/len(all_species):.1%})")

# Raccolgo i dati sui rating per le classi rare
rare_class_ratings = []
for species in rare_species:
    species_df = train_df[train_df['primary_label'] == species]
    ratings = species_df['rating'].fillna(0).tolist()  # Sostituisco NaN con 0 (nessun rating)
    
    # Statistiche per questa specie
    rare_class_ratings.append({
        'species': species,
        'count': len(species_df),
        'avg_rating': np.mean(ratings),
        'ratings': ratings,
        'rating_counts': {r: ratings.count(r) for r in set(ratings)}
    })

# Creo DataFrame per analisi
rare_ratings_df = pd.DataFrame(rare_class_ratings)
rare_ratings_df = rare_ratings_df.sort_values('count')

# Visualizzazione: Rating medi vs Conteggio per le classi rare
plt.figure(figsize=(12, 6))

# Heatmap: distribuzione dei rating per classi rare
n_rare_to_show = min(30, len(rare_ratings_df))  # Mostra max 30 classi per leggibilità
rare_sample = rare_ratings_df.head(n_rare_to_show)

# Preparo i dati per la heatmap
heatmap_data = []
rating_values = [0, 1, 2, 3, 4, 5]  # Tutti i possibili rating
for _, row in rare_sample.iterrows():
    species_data = [row['species'], row['count']]
    for rating in rating_values:
        species_data.append(row['rating_counts'].get(rating, 0))
    heatmap_data.append(species_data)

# Creo DataFrame per la heatmap
heatmap_df = pd.DataFrame(
    heatmap_data, 
    columns=['species', 'count'] + [f'rating_{r}' for r in rating_values]
)

# Plot combinato con scatter plot e heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

# Scatter plot: rating medio vs numero esempi
sns.scatterplot(
    x='count', 
    y='avg_rating', 
    data=rare_ratings_df, 
    ax=ax1, 
    alpha=0.7,
    hue='count',
    palette='viridis',
    size='count',
    sizes=(20, 200)
)
ax1.set_title('Rating Medio vs Numero di Esempi per Classi Rare')
ax1.set_xlabel('Numero di Esempi')
ax1.set_ylabel('Rating Medio')
ax1.grid(True)

# Mostra statistiche
avg_rating_rare = rare_ratings_df['avg_rating'].mean()
ax1.axhline(y=avg_rating_rare, color='r', linestyle='--', 
           label=f'Rating medio classi rare: {avg_rating_rare:.2f}')
ax1.legend()

# Heatmap: distribuzione dei rating per le classi più rare
pivot_data = pd.DataFrame({
    'species': heatmap_df['species'],
    'Rating 0': heatmap_df['rating_0'],
    'Rating 1': heatmap_df['rating_1'],
    'Rating 2': heatmap_df['rating_2'],
    'Rating 3': heatmap_df['rating_3'],
    'Rating 4': heatmap_df['rating_4'],
    'Rating 5': heatmap_df['rating_5'],
}).set_index('species')

sns.heatmap(pivot_data, cmap="YlGnBu", annot=True, fmt='g', ax=ax2)
ax2.set_title(f'Distribuzione dei Rating nelle {n_rare_to_show} Classi più Rare')

plt.tight_layout()
plt.show()

# Statistiche aggregate sui rating per le classi rare
print("\nStatistiche sui rating per le classi rare:")
print(f"- Rating medio complessivo: {rare_ratings_df['avg_rating'].mean():.2f}")
print(f"- Percentuale di registrazioni senza rating (0): {sum(r['rating_counts'].get(0, 0) for r in rare_class_ratings) / sum(r['count'] for r in rare_class_ratings):.1%}")

# Analisi delle classi estremamente rare (≤ 5 esempi)
very_rare_species = primary_species_count[primary_species_count <= 5].index.tolist()
print(f"\nClassi estremamente rare (≤ 5 esempi): {len(very_rare_species)}")

very_rare_df = train_df[train_df['primary_label'].isin(very_rare_species)]
print("Dettaglio delle registrazioni per le classi estremamente rare:")
for species in very_rare_species:
    species_data = train_df[train_df['primary_label'] == species]
    ratings = species_data['rating'].fillna(0).tolist()
    print(f"- {species}: {len(species_data)} esempi, ratings: {ratings}")


# Identificazione delle classi con solo rating molto bassi (1-2)
print("\n=== Analisi delle Classi con SOLO Rating Molto Bassi (1-2) ===")

# Funzione per verificare se una specie ha esclusivamente rating tra 1-2
def has_only_low_ratings(species_ratings):
    valid_ratings = [r for r in species_ratings if pd.notna(r) and r != 0]  # Escludi NaN e rating=0
    if not valid_ratings:  # Se non ci sono rating validi
        return False
    return all(1 <= r <= 2 for r in valid_ratings)  # Modificato: ora solo 1-2

# Raggruppa per specie e analizza
low_rating_species = []
for species, group in train_df.groupby('primary_label'):
    ratings = group['rating'].tolist()
    if has_only_low_ratings(ratings):
        low_rating_species.append({
            'species': species,
            'count': len(ratings),
            'avg_rating': np.nanmean([r for r in ratings if pd.notna(r) and r != 0]),
            'ratings': sorted([r for r in ratings if pd.notna(r) and r != 0])
        })

# Crea DataFrame e visualizza risultati
if low_rating_species:
    low_rating_df = pd.DataFrame(low_rating_species).sort_values('count', ascending=False)
    
    print(f"Trovate {len(low_rating_species)} classi con SOLO rating molto bassi (1-2):")
    
    # Visualizzazione
    plt.figure(figsize=(12, 6))
    plt.bar(low_rating_df['species'], low_rating_df['avg_rating'], alpha=0.7, color='tomato')
    plt.axhline(y=1.5, color='r', linestyle='--', label='Media teorica = 1.5')
    plt.title('Classi con Esclusivamente Rating Molto Bassi (1-2)')
    plt.xlabel('Specie')
    plt.ylabel('Rating Medio')
    plt.xticks(rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Mostra dettagli delle prime 10 classi
    print("\nDettagli delle prime 10 classi con solo rating molto bassi:")
    for i, row in low_rating_df.head(10).iterrows():
        print(f"- {row['species']}: {row['count']} clip, rating medio: {row['avg_rating']:.2f}, ratings: {row['ratings']}")
    
    # Cerca sovrapposizione con classi rare
    overlap = [s for s in low_rating_df['species'] if s in rare_species]
    print(f"\nSovrapposizione con classi rare (<{RARE_CLASS_THRESHOLD} esempi): {len(overlap)} classi")
    if overlap:
        print(f"Le classi rare che hanno solo rating molto bassi: {overlap[:10]}{'...' if len(overlap) > 10 else ''}")
else:
    print("Nessuna classe ha esclusivamente rating da 1 a 2.")

print("\n=== Conclusioni dall'Analisi delle Classi Rare ===")
print(f"1. Abbiamo {len(rare_species)} classi rare (<{RARE_CLASS_THRESHOLD} esempi)")
print(f"2. Di queste, {len(very_rare_species)} hanno 5 o meno esempi")
print("3. La qualità delle registrazioni (rating) è un fattore critico per le classi rare")
print("4. Le classi estremamente rare richiedono tecniche speciali (data augmentation, few-shot learning)")

## Data Augmentation: Implementazione delle Tecniche dei Vincitori

In questa sezione implementiamo le tre tecniche di data augmentation che hanno contribuito significativamente alle performance dei vincitori:
1. **Random Segment Selection** - Estrae segmenti casuali dalle registrazioni audio
2. **XY Masking** - Applica maschere casuali sugli assi tempo e frequenza degli spettrogrammi Mel
3. **Horizontal CutMix** - Combina parti di spettrogrammi da diverse registrazioni

La classe `AudioAugmentations` gestisce tutte queste trasformazioni in modo unificato.

In [ ]:
class AudioAugmentations:
    def __init__(self, p_random_segment=0.5, p_xy_mask=0.5, p_horizontal_cutmix=0.25):
        """
        Inizializza le trasformazioni per data augmentation audio.
        
        Args:
            p_random_segment: Probabilità di utilizzare un segmento casuale
            p_xy_mask: Probabilità di applicare il mascheramento XY
            p_horizontal_cutmix: Probabilità di applicare horizontal cutmix
        """
        self.p_random_segment = p_random_segment
        self.p_xy_mask = p_xy_mask
        self.p_horizontal_cutmix = p_horizontal_cutmix
    
    def apply_xy_masking(self, spec):
        """Applica maschere casuali sull'asse X (tempo) e Y (frequenza) allo spettrogramma"""
        mask = spec.clone()
        height, width = mask.shape[-2], mask.shape[-1]
        
        # Masking temporale (asse X)
        if np.random.random() < self.p_xy_mask:
            mask_width = int(width * np.random.uniform(0.1, 0.2))  # 10-20% width
            mask_start = np.random.randint(0, width - mask_width)
            mask[..., mask_start:mask_start+mask_width] = 0
            
        # Masking frequenziale (asse Y)
        if np.random.random() < self.p_xy_mask:
            mask_height = int(height * np.random.uniform(0.1, 0.2))  # 10-20% height
            mask_start = np.random.randint(0, height - mask_height)
            mask[:, :, mask_start:mask_start+mask_height, :] = 0
            
        return mask

### Horizontal CutMix: Implementazione della Funzione di Collate

Questa funzione personalizzata viene utilizzata nel DataLoader per implementare l'Horizontal CutMix,
che combina sezioni temporali di spettrogrammi diversi all'interno dello stesso batch.
Le etichette vengono miscelate proporzionalmente alla quantità di dati combinati.

In [ ]:
def horizontal_cutmix_collate(batch, p_cutmix=0.25):
    """
    Collate function che applica horizontal cutmix tra elementi del batch con probabilità p_cutmix.
    
    Args:
        batch: Lista di tuple (input, target)
        p_cutmix: Probabilità di applicare cutmix ad ogni coppia di esempi
    
    Returns:
        tuple: (inputs_batch, targets_batch)
    """
    inputs = []
    targets = []
    
    # Estrae input e target dal batch
    for input_tensor, target_tensor in batch:
        inputs.append(input_tensor)
        targets.append(target_tensor)
    
    inputs = torch.stack(inputs)
    targets = torch.stack(targets)
    batch_size = len(inputs)
    
    # Applica horizontal cutmix
    if batch_size > 1:  # Serve almeno 2 elementi per fare cutmix
        for i in range(batch_size):
            if np.random.rand() < p_cutmix:
                # Seleziona un altro esempio casuale da mixare
                j = np.random.randint(0, batch_size)
                if i != j:
                    # Determina il punto di taglio orizzontale (sulla dimensione temporale)
                    width = inputs.shape[3]
                    cut_point = np.random.randint(int(width * 0.25), int(width * 0.75))
                    
                    # Esegue il mixing
                    mix_ratio = cut_point / width
                    inputs[i, :, :, :cut_point] = inputs[j, :, :, :cut_point]
                    
                    # Mix delle etichette in proporzione al mix degli input
                    targets[i] = targets[i] * (1 - mix_ratio) + targets[j] * mix_ratio
    
    return inputs, targets

## Caricamento e pre-processing

## Preprocessing Audio con Supporto per Data Augmentation

La funzione di caricamento audio è stata modificata per supportare due modalità di estrazione:
- **Posizionale** - Estrazione da punti specifici nella registrazione ('start', 'center', 'end')
- **Casuale** - Estrazione di un segmento casuale quando `random_segment=True`

Questa implementazione permette di applicare la tecnica di Random Segment Selection come parte della data augmentation.

In [ ]:
def load_and_preprocess_audio(file_path, target_sr=config.SR, duration=config.DURATION, segment_position='center', random_segment=False):
    """
    Carica un file audio, estrae un segmento specifico e lo converte in spettrogramma Mel.
    
    Args:
        file_path: Percorso del file audio
        target_sr: Sample rate target
        duration: Durata target in secondi
        segment_position: Posizione del segmento ('start', 'center', 'end')
        
    Returns:
        numpy.ndarray: Spettrogramma Mel log-normalizzato
    """
    try:
        # Carica il file audio
        y, sr = librosa.load(file_path, sr=target_sr, mono=True)
        
        # Lunghezza target in campioni
        target_len = int(target_sr * duration)
        total_len = len(y)
        
        # Gestisci clip troppo corte
        if total_len < target_len:
            import math
            n_copy = math.ceil(target_len / total_len)
            if n_copy > 1:
                y = np.tile(y, n_copy)
            total_len = len(y)
        
        # Seleziona il segmento
        if random_segment and total_len > target_len:
            # Estrai segmento casuale
            max_start_idx = total_len - target_len
            start_idx = np.random.randint(0, max_start_idx)
        else:
            # Usa le posizioni predefinite
            if segment_position == 'start':
                start_idx = int(total_len * 0.2)
                if start_idx + target_len > total_len:
                    start_idx = max(0, total_len - target_len)
            elif segment_position == 'end':
                end_point = int(total_len * 0.8)
                start_idx = max(0, end_point - target_len)
            else:  # 'center' (default)
                start_idx = max(0, int(total_len / 2 - target_len / 2))
        
        # Estrai il segmento
        y = y[start_idx:start_idx + target_len]
        
        # Padda se necessario
        if len(y) < target_len:
            y = np.pad(y, (0, target_len - len(y)), mode='constant')
        
        # Calcola lo spettrogramma Mel
        mel_spec = librosa.feature.melspectrogram(
            y=y, sr=sr,
            n_fft=config.N_FFT,
            hop_length=config.HOP_LENGTH,
            n_mels=config.N_MELS,
            fmin=config.FMIN,
            fmax=config.FMAX,
            power=config.POWER
        )
        
        # Converti in scala logaritmica (dB)
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        
        # Normalizza
        min_val = np.min(log_mel_spec)
        max_val = np.max(log_mel_spec)
        if max_val > min_val:
            log_mel_spec = (log_mel_spec - min_val) / (max_val - min_val)
        else:
            log_mel_spec = np.zeros_like(log_mel_spec)
        
        return log_mel_spec
        
    except Exception as e:
        print(f"Errore nell'elaborazione di {file_path}: {e}")
        time_steps = int(target_sr * duration / config.HOP_LENGTH) + 1
        return np.zeros((config.N_MELS, time_steps), dtype=np.float32)

## 5. Dataset PyTorch per Dati Audio

## Dataset PyTorch con Supporto Integrato per Data Augmentation

Implementiamo due classi di dataset:
- `BirdDataset` - Dataset standard che estrae il segmento centrale delle clip audio
- `AdaptiveMultiSegmentBirdDataset` - Dataset avanzato che:
  - Estrae automaticamente diversi segmenti in base alla durata delle registrazioni
  - Applica le tecniche di data augmentation durante il caricamento
  - Supporta l'estrazione di segmenti adattivi (1-3 per clip in base alla lunghezza)

In [ ]:
class BirdDataset(Dataset):
    def __init__(self, df, audio_dir, labels_one_hot, transform=None):
        """
        Dataset che estrae solo il segmento centrale per ogni clip audio.
        """
        self.df = df
        self.audio_dir = audio_dir
        self.labels = labels_one_hot
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row['filename']
        file_path = os.path.join(self.audio_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"Attenzione: File non trovato in {file_path}.")
            time_steps = int(config.SR * config.DURATION / config.HOP_LENGTH) + 1
            dummy_spec = torch.zeros((1, config.N_MELS, time_steps), dtype=torch.float32)
            dummy_label = torch.zeros(config.N_CLASSES, dtype=torch.float32)
            return dummy_spec, dummy_label
            
        # Carica e preprocessa l'audio con solo segmento centrale
        mel_spec = load_and_preprocess_audio(file_path, segment_position='center')
        
        # Aggiungi dimensione del canale e converti in tensor
        mel_spec = np.expand_dims(mel_spec, axis=0)
        mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32)
        
        # Ottieni le etichette
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        if self.transform:
            mel_spec_tensor = self.transform(mel_spec_tensor)
            
        return mel_spec_tensor, label_tensor

## BirdDataset con l'utilizzo di adaptive clip in base alla lunghezza

In [ ]:
class AdaptiveMultiSegmentBirdDataset(Dataset):
    def __init__(self, df, audio_dir, labels_one_hot, transform=None, augmentations=None):
        """
        Dataset che estrae un numero appropriato di segmenti in base alla lunghezza di ogni clip.
        
        Args:
            df: DataFrame con i metadati
            audio_dir: Directory contenente i file audio
            labels_one_hot: Array di etichette one-hot
            transform: Trasformazioni da applicare (opzionale)
        """
        self.df = df
        self.audio_dir = audio_dir
        self.labels = labels_one_hot
        self.transform = transform
        self.augmentations = augmentations
        
        # Pre-calcola quali segmenti usare per ogni clip
        self.segments_to_use = []
        print("Analizzando le lunghezze delle clip audio...")
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Preparazione dataset adattivo"):
            file_path = os.path.join(audio_dir, row['filename'])
            try:
                # Carica solo l'informazione sulla durata senza caricare l'intero file
                y, sr = librosa.load(file_path, sr=None, duration=0.1)  # Carica solo un breve segmento per ottenere SR
                info = librosa.get_duration(filename=file_path, sr=sr)
                total_duration = info  # Durata in secondi
                
                # Determina quali segmenti usare in base alla durata
                if total_duration < config.DURATION * 1.5:
                    # Clip troppo corta per multiple segmenti, usa solo il centro
                    segments = ['center']
                elif total_duration < config.DURATION * 2.5:
                    # Clip media, usa inizio e fine
                    segments = ['start', 'end']
                else:
                    # Clip abbastanza lunga, usa tutti e tre i segmenti
                    segments = ['start', 'center', 'end']
                
                # Memorizza l'indice originale e i segmenti da utilizzare
                for segment in segments:
                    self.segments_to_use.append((idx, segment))
                    
            except Exception as e:
                # In caso di errore, usa solo il segmento centrale
                self.segments_to_use.append((idx, 'center'))
                print(f"Errore nell'elaborazione di {file_path}: {e}")
    
    def __len__(self):
        return len(self.segments_to_use)
    
    def __getitem__(self, idx):
        df_idx, segment_position = self.segments_to_use[idx]
        
        # Ottieni il record dal DataFrame originale
        row = self.df.iloc[df_idx]
        filename = row['filename']
        file_path = os.path.join(self.audio_dir, filename)
        
        # Determina se usare un segmento casuale (random segment augmentation)
        use_random_segment = self.augmentations and np.random.random() < self.augmentations.p_random_segment
        
        # Carica e preprocessa l'audio con il segmento selezionato o casuale
        mel_spec = load_and_preprocess_audio(
            file_path, 
            segment_position=segment_position,
            random_segment=use_random_segment
        )
        
        # Aggiungi dimensione del canale e converti in tensor
        mel_spec = np.expand_dims(mel_spec, axis=0)
        mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32)
        
        # Applica XY masking se le augmentations sono attive
        if self.augmentations and np.random.random() < self.augmentations.p_xy_mask:
            mel_spec_tensor = self.augmentations.apply_xy_masking(mel_spec_tensor)
        
        # Applica altre trasformazioni se necessario
        if self.transform:
            mel_spec_tensor = self.transform(mel_spec_tensor)
            
        # Ottieni le etichette corrispondenti
        label_tensor = torch.tensor(self.labels[df_idx], dtype=torch.float32)
            
        return mel_spec_tensor, label_tensor

### Crezione dataset e dataloader con AdaptiveMultiSegment

### Creazione dei DataLoader con Augmentation

In questa sezione:
1. Applichiamo il bilanciamento strategico al dataset di training
2. Creiamo l'istanza di AudioAugmentations con le probabilità ottimali
3. Configuriamo i dataloader con le funzioni di collate personalizzate
4. Analizziamo la distribuzione dei segmenti nel dataset risultante

In [ ]:
# Applica il bilanciamento strategico solo al dataset di training
print("\n=== Bilanciamento Strategico del Dataset di Training ===")
X_train_df_balanced, y_train_one_hot_balanced = create_balanced_dataset_df(
    X_train_df, 
    y_train_one_hot,
    abundant_class_threshold=150,  # Classi con più di 150 esempi sono considerate abbondanti
    remove_percentage=0.4  # Rimuove il 40% degli esempi con rating bassi
)

# Crea un'istanza delle augmentations audio
audio_augmentations = AudioAugmentations(p_random_segment=0.5, p_xy_mask=0.5, p_horizontal_cutmix=0.25)

# Creiamo i dataset utilizzando AdaptiveMultiSegmentBirdDataset per il training
print("Creazione dataset di training con approccio multi-segmento adattivo...")
# Creiamo i dataset utilizzando AudioAugmentations
train_dataset = AdaptiveMultiSegmentBirdDataset(
    X_train_df_balanced, 
    config.TRAIN_AUDIO_DIR, 
    y_train_one_hot_balanced,
    augmentations=audio_augmentations  # Aggiungi augmentations
)
# Per validation e test, usiamo il dataset standard con segmento centrale
print("Creazione dataset di validation con segmento centrale...")
# Per validation, non usiamo augmentation
val_dataset = BirdDataset(X_val_df, config.TRAIN_AUDIO_DIR, y_val_one_hot)

# Stampa informazioni sulla dimensione effettiva del dataset
print(f"\nNumero di record originali nel training set: {len(X_train_df)}")
print(f"Numero di campioni effettivi nel training set dopo l'adattamento: {len(train_dataset)}")
print(f"Rapporto di espansione: {len(train_dataset) / len(X_train_df):.2f}x")

# Analizza la distribuzione dei tipi di segmenti
segment_types = [segment for _, segment in train_dataset.segments_to_use]
segment_counts = {
    'start': segment_types.count('start'),
    'center': segment_types.count('center'),
    'end': segment_types.count('end')
}

print("\nDistribuzione dei segmenti nel dataset:")
for segment, count in segment_counts.items():
    print(f"- {segment}: {count} ({count/len(train_dataset):.1%})")

# Creiamo i dataloader con horizontal cutmix per il training
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=True,
    num_workers=config.NUM_WORKERS, 
    pin_memory=True,
    collate_fn=lambda batch: horizontal_cutmix_collate(batch, p_cutmix=0.25)  # Custom collate
)

# Per validation, non usiamo cutmix
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=False,
    num_workers=config.NUM_WORKERS, 
    pin_memory=True
)

# Non creiamo un test_loader per ora
test_loader = None
    
print(f"Numero di batch di training per epoca: {len(train_loader)}")
print(f"Numero di batch di validation per epoca: {len(val_loader)}")
print("Test set: utilizzerò direttamente i file nella cartella test_soundscapes")

## 6. Definizione del Modello CNN

## Modello EfficientNet con Head Personalizzata

Implementiamo un modello basato su EfficientNet-B0 preaddestrato, con:
- Supporto per input a singolo canale (spettrogrammi Mel)
- Testa di classificazione personalizzata con dropout e normalizzazione batch
- Parametri differenziati per l'ottimizzazione
- Gestione automatica dei checkpoint e dei pesi preaddestrati

In [ ]:
class EfficientNetBirdClassifier(nn.Module):
    def __init__(self, num_classes=config.N_CLASSES, pretrained=True, model_name='efficientnet_b0'):
        super(EfficientNetBirdClassifier, self).__init__()
        
        # Tenta di caricare il modello con pesi pre-addestrati, gestendo fallimenti di connessione
        try:
            if pretrained:
                print(f"Tentativo di caricare {model_name} con pesi pre-addestrati...")
                self.efficientnet = timm.create_model(
                    model_name,
                    pretrained=True,
                    num_classes=0  # Rimuovi il classificatore originale
                )
                print("Modello caricato con successo con pesi pre-addestrati.")
            else:
                # Se pretrained=False, non tentare di scaricare i pesi
                self.efficientnet = timm.create_model(
                    model_name,
                    pretrained=False,  # CORRETTO: ora è False
                    num_classes=0  # Rimuovi il classificatore originale
                )
                print("Modello inizializzato senza pesi pre-addestrati (modalità offline).")
        except Exception as e:
            print(f"Errore nel caricamento dei pesi pre-addestrati: {e}")
            print("Inizializzazione del modello senza pesi pre-addestrati...")
            self.efficientnet = timm.create_model(
                model_name,
                pretrained=False,
                num_classes=0  # Rimuovi il classificatore originale
            )
        
        # Ottieni la dimensione dell'output del feature extractor
        if hasattr(self.efficientnet, 'num_features'):
            classifier_in_features = self.efficientnet.num_features
        elif hasattr(self.efficientnet, 'classifier'):
            classifier_in_features = self.efficientnet.classifier.in_features
        else:
            # Valore predefinito per EfficientNet-B0
            classifier_in_features = 1280
        
         # Sostituisci il classificatore semplice con una MLP con dropout
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),  # Primo dropout significativo
            nn.Linear(classifier_in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),  # Secondo dropout più leggero
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        # Se l'input è un'immagine a 1 canale, replicala su 3 canali
        if x.size(1) == 1:
            x = x.repeat(1, 3, 1, 1)
        
        # Passa l'input attraverso il backbone per ottenere le features
        features = self.efficientnet(x)
        
        # Passa le feature attraverso il classificatore
        output = self.classifier(features)
        
        return output

# Controlla se esiste un checkpoint precedente
has_previous_checkpoint = False
if config.environment == 'kaggle':
    latest_checkpoint = '/kaggle/working/checkpoints/latest_checkpoint.pth'
    has_previous_checkpoint = os.path.exists(latest_checkpoint)
elif config.environment == 'colab':
    drive_checkpoint = '/content/drive/MyDrive/birdclef_checkpoints/latest_checkpoint.pth'
    has_previous_checkpoint = os.path.exists(drive_checkpoint)
else:
    # Per ambienti locali
    local_checkpoint = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'latest_checkpoint.pth')
    has_previous_checkpoint = os.path.exists(local_checkpoint)

# Inizializza il modello - usa pretrained=False se hai già un checkpoint
model = EfficientNetBirdClassifier(
    num_classes=config.N_CLASSES, 
    pretrained=not has_previous_checkpoint  # Scarica i pesi solo se non c'è già un checkpoint
).to(config.DEVICE)

# Definiamo ottimizzatore con learning rate differenziati
def get_optimizer(model, lr_base=5e-4, lr_head=1e-3):  # Ridotti significativamente
    # Parametri del corpo (pre-addestrati)
    backbone_params = [p for name, p in model.named_parameters() 
                      if 'classifier' not in name]
    
    # Parametri della testa (da addestrare da zero)
    classifier_params = [p for name, p in model.named_parameters() 
                      if 'classifier' in name]
    
    optimizer = optim.AdamW([
        {'params': backbone_params, 'lr': lr_base},  # Ridotto da 1e-3 a 5e-4
        {'params': classifier_params, 'lr': lr_head}  # Ridotto da 3e-3 a 1e-3
    ], weight_decay=3e-4)
    
    return optimizer

# Loss function - manteniamo BCE come richiesto
criterion = nn.BCEWithLogitsLoss()

# Ottimizzatore con learning rate differenziati
optimizer = get_optimizer(model)

# Learning rate scheduler - aggiornato a CosineAnnealingLR come usato dai vincitori
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config.EPOCHS,  # Numero totale di epoche
    eta_min=1e-6  # Learning rate minimo
)

print(f"Modello EfficientNet-B0 (timm) caricato su {config.DEVICE}")
print(f"Numero di classi: {config.N_CLASSES}")
print(f"Parametri totali: {sum(p.numel() for p in model.parameters()):,}")

## 7. Addestramento e Validazione del Modello

## Funzione di Training con Supporto per Checkpoint

La funzione di training implementa:
- Caricamento automatico dei checkpoint precedenti
- Early stopping basato sulle performance di validation
- Salvataggio periodico dei checkpoint e del miglior modello
- Supporto per scheduler di learning rate (CosineAnnealingLR)
- Visualizzazione delle curve di loss

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                epochs=config.EPOCHS, device=config.DEVICE, 
                model_save_path=None, model_load_path=None, patience=3,
                resume_training=True, scheduler=None):
    """
    Addestra il modello e valuta su validation set con supporto per checkpoint.
    
    Args:
        model: Modello PyTorch da addestrare
        train_loader: DataLoader per dati di training
        val_loader: DataLoader per dati di validation
        criterion: Funzione di loss
        optimizer: Ottimizzatore
        epochs: Numero di epoche di training
        device: Device per l'addestramento ('cuda' o 'cpu')
        model_save_path: Path dove salvare il modello addestrato
        model_load_path: Path da cui caricare un modello pre-addestrato
        patience: Numero di epoche senza miglioramento prima di terminare l'addestramento
        resume_training: Se True, riprende il training da un checkpoint (se disponibile)
        scheduler: Learning rate scheduler
        
    Returns:
        tuple: (train_losses, val_losses, total_training_time)
    """
    # Directory per i checkpoint in base all'ambiente
    checkpoint_dir = None
    drive_mounted = False
    
    # Configura la directory per i checkpoint a seconda dell'ambiente
    if config.environment == 'colab':
        try:
            from google.colab import drive
            # Controlla se il drive è già montato
            if not os.path.exists('/content/drive'):
                print("Montaggio di Google Drive...")
                drive.mount('/content/drive')
                print("Google Drive montato con successo.")
            
            # Crea directory per i checkpoint se non esiste
            checkpoint_dir = '/content/drive/MyDrive/birdclef_checkpoints'
            os.makedirs(checkpoint_dir, exist_ok=True)
            print(f"Directory per i checkpoint creata su Google Drive: {checkpoint_dir}")
            
            # Aggiorna il percorso di salvataggio per usare Google Drive
            if model_save_path:
                filename = os.path.basename(model_save_path)
                model_save_path = os.path.join(checkpoint_dir, filename)
                print(f"Il modello sarà salvato in: {model_save_path}")
            
            drive_mounted = True
        except ImportError:
            print("Errore: Non riesco ad accedere a Google Drive. Continuo senza persistenza.")
        except Exception as e:
            print(f"Errore durante il montaggio di Google Drive: {e}")
            print("Continuo senza persistenza su Drive.")
    elif config.environment == 'kaggle':
        # In Kaggle, usa la directory di working
        checkpoint_dir = '/kaggle/working/checkpoints'
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Directory per i checkpoint creata in Kaggle: {checkpoint_dir}")
    else:
        # In locale, usa la directory 'checkpoints' nell'OUTPUT_DIR
        checkpoint_dir = os.path.join(config.OUTPUT_DIR, 'checkpoints')
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Directory per i checkpoint creata in locale: {checkpoint_dir}")
    
    # Inizializzazione variabili
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    total_training_time = 0
    start_epoch = 0
    needs_training = True
    checkpoint_exists = False
    model_loaded = False
    
    # Verifica se esiste un modello pre-addestrato da caricare
    if model_load_path and os.path.exists(model_load_path):
        print(f"Modello trovato in {model_load_path}. Tentativo di caricamento...")
        try:
            checkpoint = torch.load(model_load_path, map_location=device)
            
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                model.load_state_dict(checkpoint)
                
            print("Modello caricato con successo.")
            model_loaded = True
            needs_training = False
        except Exception as e:
            print(f"Errore durante il caricamento del modello: {e}")
            print("Verrà avviato l'addestramento da zero.")
            needs_training = True
    else:
        print(f"Modello non trovato in {model_load_path}.")
    
    # Cerca un checkpoint SOLO se il caricamento del modello è fallito E resume_training è True
    if needs_training and resume_training and checkpoint_dir and not model_loaded:
        latest_checkpoint = os.path.join(checkpoint_dir, "latest_checkpoint.pth")
        if os.path.exists(latest_checkpoint):
            print(f"Trovato checkpoint in {latest_checkpoint}. Tentativo di caricamento...")
            try:
                checkpoint = torch.load(latest_checkpoint, map_location=device)
                
                # Verifica che sia un checkpoint compatibile prima di caricarlo
                if isinstance(checkpoint, dict) and 'epoch' in checkpoint:
                    try:
                        model.load_state_dict(checkpoint['model_state_dict'])
                        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        start_epoch = checkpoint['epoch'] + 1
                        train_losses = checkpoint['train_losses']
                        val_losses = checkpoint['val_losses']
                        best_val_loss = checkpoint['best_val_loss']
                        epochs_without_improvement = checkpoint['epochs_without_improvement']
                        total_training_time = checkpoint.get('total_training_time', 0)
                        
                        # Ricrea lo scheduler con lo stato salvato se presente
                        if scheduler is not None and 'scheduler_state_dict' in checkpoint:
                            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                        
                        print(f"Checkpoint caricato con successo (epoca {start_epoch-1})")
                        print(f"Si riparte dall'epoca {start_epoch}/{epochs}")
                        
                        if start_epoch >= epochs:
                            needs_training = False
                        
                        checkpoint_exists = True
                    except Exception as e:
                        print(f"Il checkpoint non è compatibile con il modello attuale: {e}")
                        print("Verrà avviato l'addestramento da zero.")
            except Exception as e:
                print(f"Errore durante il caricamento del checkpoint: {e}")
                print("Si procederà con il training da zero.")
    
    model.to(device)
    
    # Esegui training solo se necessario
    if needs_training:
        start_time_total = time.time()
        model.train()
        
        # Loop di training sulle epoche (inizia da start_epoch)
        for epoch in range(start_epoch, epochs):
            epoch_start_time = time.time()
            
            # --- Fase di Training ---
            model.train()
            running_loss = 0.0
            pbar_train = tqdm(enumerate(train_loader), total=len(train_loader), 
                             desc=f"Epoca {epoch+1}/{epochs} [Train]", leave=True)
            
            for i, (inputs, labels) in pbar_train:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                avg_loss = running_loss / (i + 1)
                pbar_train.set_postfix({'loss': f"{avg_loss:.4f}"})
            
            epoch_train_loss = running_loss / len(train_loader)
            train_losses.append(epoch_train_loss)
            
            # --- Fase di Validation ---
            model.eval()
            running_val_loss = 0.0
            pbar_val = tqdm(enumerate(val_loader), total=len(val_loader), 
                           desc=f"Epoca {epoch+1}/{epochs} [Val]", leave=True)
            
            with torch.no_grad():
                for i, (val_inputs, val_labels) in pbar_val:
                    val_inputs = val_inputs.to(device)
                    val_labels = val_labels.to(device)
                    
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    avg_val_loss = running_val_loss / (i + 1)
                    pbar_val.set_postfix({'val_loss': f"{avg_val_loss:.4f}"})
            
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            
            # Aggiornamento scheduler - modificato per CosineAnnealingLR
            if scheduler is not None:
                if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(epoch_val_loss)  # Per ReduceLROnPlateau
                else:
                    scheduler.step()  # Per CosineAnnealingLR è step() senza parametri
            
            epoch_end_time = time.time()
            epoch_duration = epoch_end_time - epoch_start_time
            total_training_time += epoch_duration
            
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, "
                  f"Val Loss: {epoch_val_loss:.4f}, Duration: {epoch_duration:.2f} sec")
            
            # Salvataggio checkpoint per ogni epoca (in qualsiasi ambiente)
            if checkpoint_dir:
                checkpoint_path = os.path.join(checkpoint_dir, f"birdclef_epoch_{epoch+1}.pth")
                
                # Salva checkpoint completo con tutte le informazioni di stato
                checkpoint = {
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'best_val_loss': best_val_loss,
                    'epochs_without_improvement': epochs_without_improvement,
                    'total_training_time': total_training_time
                }
                
                # Salva anche lo stato dello scheduler se esiste
                if scheduler is not None:
                    checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                
                torch.save(checkpoint, checkpoint_path)
                print(f"Checkpoint completo salvato in {checkpoint_path}")
                
                # Aggiorna anche il checkpoint più recente (sovrascrive)
                torch.save(checkpoint, os.path.join(checkpoint_dir, "latest_checkpoint.pth"))
            
            # Early stopping
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                epochs_without_improvement = 0
                # Salva il miglior modello separatamente
                if model_save_path:
                    best_path = model_save_path.replace('.pth', '_best.pth')
                    
                    # Salva checkpoint completo
                    best_checkpoint = {
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_losses': train_losses,
                        'val_losses': val_losses,
                        'best_val_loss': best_val_loss
                    }
                    
                    # Salva anche lo stato dello scheduler
                    if scheduler is not None:
                        best_checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                    
                    torch.save(best_checkpoint, best_path)
                    print(f"Salvato miglior modello in {best_path}")
            else:
                epochs_without_improvement += 1
                
            if epochs_without_improvement >= patience:
                print(f"\nEarly stopping attivato! Nessun miglioramento per {patience} epoche consecutive.")
                break
        
        end_time_total = time.time()
        if checkpoint_exists:
            total_training_time += (end_time_total - start_time_total)
        else:
            total_training_time = end_time_total - start_time_total
            
        print(f"\nTraining terminato in {total_training_time/60:.2f} minuti totali")
        
        # Salva il modello finale
        if model_save_path:
            final_checkpoint = {
                'epoch': epochs-1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_losses': train_losses,
                'val_losses': val_losses,
                'best_val_loss': best_val_loss,
                'total_training_time': total_training_time
            }
            
            # Salva anche lo stato dello scheduler
            if scheduler is not None:
                final_checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                
            torch.save(final_checkpoint, model_save_path)
            print(f"Modello finale salvato in {model_save_path}")
    else:
        print("Training non necessario: modello già caricato o training ripreso e completato.")
    
    # Visualizza le curve di loss
    if train_losses and val_losses:
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
        plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoche')
        plt.ylabel('Loss')
        plt.title('Curve di Loss di Training e Validation')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Salva il grafico
        if checkpoint_dir:
            plt_path = os.path.join(checkpoint_dir, 'loss_curves.png')
            plt.savefig(plt_path)
            print(f"Grafico delle curve di loss salvato in {plt_path}")
    
    return train_losses, val_losses, total_training_time

### Configurazione e Avvio del Training

Configuriamo e avviamo il training del modello:
- Individuazione automatica dei checkpoint precedenti
- Inizializzazione dell'ottimizzatore e dello scheduler
- Avvio del training con i parametri ottimizzati

In [ ]:
# Percorsi per caricamento/salvataggio del modello
if config.environment == 'kaggle':
    # Directory per i checkpoint in Kaggle
    os.makedirs('/kaggle/working/checkpoints', exist_ok=True)
    
    # Verifica se esiste un checkpoint precedente
    latest_checkpoint = '/kaggle/working/checkpoints/latest_checkpoint.pth'
    if os.path.exists(latest_checkpoint):
        model_load_path = latest_checkpoint
        print(f"Trovato checkpoint precedente in {latest_checkpoint}")
    else:
        # Usa un modello base precaricato se disponibile
        model_load_path = "/kaggle/input/baseline_10_epoch_clip_adaptive_efficientNet/pytorch/default/1/birdclef_trained_model.pth"
        
    # Imposta il percorso di salvataggio
    model_save_path = "/kaggle/working/birdclef_trained_model_efficientNET_timm.pth"
    
elif config.environment == 'colab':
    # Per Colab, verifica se esiste un checkpoint su Drive
    drive_checkpoint = '/content/drive/MyDrive/birdclef_checkpoints/latest_checkpoint.pth'
    if os.path.exists(drive_checkpoint):
        model_load_path = drive_checkpoint
        print(f"Trovato checkpoint precedente su Drive: {drive_checkpoint}")
    else:
        # Altrimenti usa un modello base se disponibile
        model_load_path = os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth") if os.path.exists(os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth")) else None
    
    model_save_path = os.path.join(config.OUTPUT_DIR, f"birdclef_model_timm_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")
    
else:
    # Per ambienti locali
    local_checkpoint = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'latest_checkpoint.pth')
    if os.path.exists(local_checkpoint):
        model_load_path = local_checkpoint
        print(f"Trovato checkpoint precedente: {local_checkpoint}")
    else:
        model_load_path = os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth") if os.path.exists(os.path.join(config.MODELS_DIR, "baseline_bird_cnn_model_val.pth")) else None
    
    model_save_path = os.path.join(output_dirs['checkpoints'], f"birdclef_model_timm_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")

# Addestra il modello con CosineAnnealingLR scheduler
train_losses, val_losses, training_time = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,  # Passa lo scheduler
    epochs=config.EPOCHS,
    device=config.DEVICE,
    model_save_path=model_save_path,
    model_load_path=model_load_path,
    resume_training=True
)

## 8. Generazione della Submission

## Generazione della Submission

Implementiamo la funzione per generare le predizioni sui file di test:
- Caricamento e segmentazione delle soundscape di test
- Estrazione di spettrogrammi Mel da ciascun segmento
- Generazione delle predizioni con il modello addestrato
- Creazione del file di submission nel formato richiesto dalla competizione

In [ ]:
def generate_submission_simple(model, device=config.DEVICE):
    """
    Genera un file di submission usando una segmentazione semplificata.
    
    Args:
        model: Modello PyTorch addestrato
        device: Device per inferenza ('cuda' o 'cpu')
        
    Returns:
        pd.DataFrame: DataFrame di submission
    """
    model.to(device)
    model.eval()
    
    # Set seed per riproducibilità
    np.random.seed(42)
    
    # Percorso dei test soundscapes
    test_soundscape_path = config.TEST_SOUNDSCAPES_DIR
    test_soundscapes = [os.path.join(test_soundscape_path, afile) 
                        for afile in sorted(os.listdir(test_soundscape_path)) 
                        if afile.endswith('.ogg')]
    
    print(f"Elaborazione di {len(test_soundscapes)} file soundscape...")
    
    # Crea DataFrame per le predizioni
    predictions = pd.DataFrame(columns=['row_id'] + all_species)
    
    for soundscape in tqdm(test_soundscapes, desc="Elaborazione soundscapes"):
        # Carica audio
        sig, rate = librosa.load(path=soundscape, sr=config.SR)
        
        # Split in segmenti da 5 secondi
        segment_length = rate * config.TEST_CLIP_DURATION
        chunks = []
        for i in range(0, len(sig), segment_length):
            chunk = sig[i:i+segment_length]
            # Padda se necessario
            if len(chunk) < segment_length:
                chunk = np.pad(chunk, (0, segment_length - len(chunk)), mode='constant')
            chunks.append(chunk)
        
        # Genera predizioni per ogni segmento
        for i, chunk in enumerate(chunks):
            # Calcola row_id (nome file + tempo finale del segmento in secondi)
            file_name = os.path.basename(soundscape).split('.')[0]
            row_id = f"{file_name}_{i * config.TEST_CLIP_DURATION + config.TEST_CLIP_DURATION}"
            
            # Calcola spettrogramma Mel
            mel_spec = librosa.feature.melspectrogram(
                y=chunk, sr=config.SR,
                n_fft=config.N_FFT,
                hop_length=config.HOP_LENGTH,
                n_mels=config.N_MELS,
                fmin=config.FMIN,
                fmax=config.FMAX
            )
            
            # Converti in scala logaritmica (dB) e normalizza
            log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            min_val = np.min(log_mel_spec)
            max_val = np.max(log_mel_spec)
            if max_val > min_val:
                log_mel_spec = (log_mel_spec - min_val) / (max_val - min_val)
            else:
                log_mel_spec = np.zeros_like(log_mel_spec)
            
            # Aggiungi dimensione batch e canale
            log_mel_spec = np.expand_dims(np.expand_dims(log_mel_spec, axis=0), axis=0)
            
            # Converti in tensor
            input_tensor = torch.tensor(log_mel_spec, dtype=torch.float32).to(device)
            
            # Effettua predizione
            with torch.no_grad():
                output = model(input_tensor)
                scores = torch.sigmoid(output).cpu().numpy()[0]
            
            # Aggiungi riga al DataFrame di predizioni
            new_row = pd.DataFrame([[row_id] + list(scores)], columns=['row_id'] + all_species)
            predictions = pd.concat([predictions, new_row], axis=0, ignore_index=True)
    
    # Salva la submission come CSV
    predictions.to_csv("submission.csv", index=False)
    
    return predictions

# Genera submission
if config.environment == 'kaggle':
    print("\nGenerazione del file di submission con il nuovo metodo...")
    submission_df = generate_submission_simple(model)
    
    if submission_df is not None:
        print("\nAnteprima del file di submission:")
        print(submission_df.head())
else:
    print("\nSalto la generazione della submission perché non siamo su Kaggle.")